In [1]:
import dill
import mne
import numpy as np

In [20]:
n_channels = 27

fwd = mne.read_forward_solution(f'leadfield/{n_channels}_20k_fwd.fif')

Reading forward solution from /home/hareen/Projects/bunny/leadfield/27_20k_fwd.fif...
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read EEG forward solution (20484 sources, 27 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame


In [16]:
data = mne.io.read_raw_eeglab('oddball/sub-001/eeg/sub-001_task-P300_run-1_eeg.set')
remapping = {k: k.upper().replace("Z", "z").replace("FP", "Fp") for k in data.ch_names}
data = data.rename_channels(remapping)

for ch in fwd.ch_names:
    if not ch in data.ch_names:
        data = mne.add_reference_channels(data, ch)

data = data.pick(fwd.ch_names)

/tmp/ipykernel_3289/2398652722.py:1: RuntimeWarning: Unknown types found, setting as type EEG:
temp: ['Temp']
  data = mne.io.read_raw_eeglab('oddball/sub-001/eeg/sub-001_task-P300_run-1_eeg.set')
/tmp/ipykernel_3289/2398652722.py:1: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  data = mne.io.read_raw_eeglab('oddball/sub-001/eeg/sub-001_task-P300_run-1_eeg.set')
/tmp/ipykernel_3289/2398652722.py:1: RuntimeWarning: Not setting positions of 14 gsr/misc/resp channels found in montage:
['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  data = mne.io.read_raw_eeglab('oddball/sub-001/eeg/sub-001_task-P300_run-1_eeg.set')
/tmp/ipykernel_3289/2398652722.py:7: Runtime

In [21]:
data = mne.io.read_raw_edf('epilepsy_27/epilepsy_27.edf', preload=True)
remapping = {k: k.split(' ')[1].upper().replace("Z", "z").replace("FP", "Fp") for k in data.ch_names}
data = data.rename_channels(remapping)

for ch in fwd.ch_names:
    if not ch in data.ch_names:
        data = mne.add_reference_channels(data, ch)

data = data.pick(fwd.ch_names)

Extracting EDF parameters from /home/hareen/Projects/bunny/epilepsy_27/epilepsy_27.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


In [42]:
eeg = data.get_data()[:, :10000]
eeg -= np.mean(eeg, axis=0, keepdims=True)
eeg -= np.mean(eeg, axis=1, keepdims=True)
eeg /= np.abs(eeg).max()

while eeg.shape[1] < 500:
    eeg = np.concatenate((eeg, eeg), axis=1)

In [43]:
with open('source/eeg/27_spike_sample2.pkl', 'wb') as f:
    dill.dump(eeg, f)